RAG是给LLMs上外挂知识库，解决的是幻觉、实时性的问题，Agent则是给LLMs外挂工具，解决的是智能化流程、复杂问题的解决能力

这段脚本展示了如何使用 LlamaIndex 和相关工具构建一个智能代理（Agent），该代理能够利用外部知识库和各种功能工具来解决复杂的问题。通过设置语言模型、加载文档数据构建向量索引，以及定义具体的工具函数，该代理可以进行对话交互，并根据问题的需要调用不同的工具和查询外部信息。最终，这个代理能够在一系列提示词的引导下，像一个具有思维链能力的智能系统一样，逐步解决问题并提供答案。

总得来说，Agent=LLMs+记忆+工具，通过LLMs类似思维链的方式根据工具生成解决问题的步骤，然后根据工具调用结果判断下一步，直到找到问题的答案

```mermaid
flowchart LR
    A[普通函数]
    B["转为tools
       FunctionTool.from_defaults"];
    C["llamaIndex自带tools
      llama_index.tools.tavily_research"]
    D["将RAG查询引擎转为tools
        QueryEngineTool.from_defaults"]
    E[工具列表tools]
    F["通过工具创建Agent
      ReActAgent.from_tools"]
    A ---> B;
    B---> E;
    C--->E;
    D--->E;
    E--->F;
```

代码总结：

- 使用`ReActAgent.from_tools()`方法创建代理实例。该方法接受一个工具列表作为参数，这些工具可能包括数学计算、外部API调用等。
- `QueryEngineTool.from_defaults(query_engine)`将查询引擎封装成可以直接使用的工具，并赋予其名称和描述，方便代理在需要时选择使用。

In [1]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
base_url='http://localhost:11434'
Settings.llm = Ollama(model="qwen2.5:latest", request_timeout=360.0,base_url=base_url)
Settings.embed_model = OllamaEmbedding(model_name="quentinz/bge-large-zh-v1.5:latest",base_url=base_url)


In [2]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b
multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b
add_tool = FunctionTool.from_defaults(fn=add)

# 创建代理 
agent = ReActAgent.from_tools([multiply_tool, add_tool], verbose=True)
response = agent.chat("1000+157*2？使用工具计算每一步")
print(response)

> Running step 8966df7d-bd40-4e07-b20a-f93c1ddcd237. Step input: 1000+157*2？使用工具计算每一步
Thought: 我需要先进行乘法运算，然后进行加法运算。我将首先使用multiply工具来完成乘法。
Action: multiply
Action Input: {'a': 157, 'b': 2}
Observation: 314
> Running step d0c73742-4fb1-442c-a5b1-5db9b941d01c. Step input: None
Thought: 我已经得到了157 * 2的结果是314。接下来，我需要使用add工具来进行加法运算。
Action: add
Action Input: {'a': 1000, 'b': 314}
Observation: 1314
> Running step dd89ef73-1d9a-493b-86cc-b3189587fd81. Step input: None
Thought: 我可以回答这个问题了。我现在知道1000 + 157 * 2等于1314。
Answer: 计算结果是1314。
计算结果是1314。


实际上，agent不仅仅是文本生成，还记忆了历史聊天的过程，回答时会考虑以前的聊天内容

In [3]:
agent.memory.set([]) # 清空前一次聊天记录 
# 短期记忆
response = agent.chat('你是一名数学老师，擅长用通俗的语言解决各种数学问题，你的名字叫小数')
print('----'*5)
response = agent.chat('请问1000*2+234等于多少？')
print('----'*5)
response = agent.chat('你是谁？你能做什么？')

> Running step 5fdec7a6-3f3f-47b4-8a44-1bda9961a384. Step input: 你是一名数学老师，擅长用通俗的语言解决各种数学问题，你的名字叫小数
Thought: 我可以回答这个问题而不需要使用任何工具。我将用用户的问题中使用的语言来作答。
Answer: 你好！很高兴能以小数的身份帮助大家解答数学问题。如果你有任何数学上的疑问或是需要解释某个概念，请随时提问，我会尽力用通俗易懂的语言来解答哦！
--------------------
> Running step 1d018ddc-98c2-4b42-ba85-5da712b0299c. Step input: 请问1000*2+234等于多少？
Thought: 我需要用工具来计算这个表达式的值。
Action: multiply
Action Input: {'a': 1000.0, 'b': 2.0}
Observation: 2000.0
> Running step 1f514a95-349f-4acb-8106-736faf823fca. Step input: None
Thought: 现在我得到了乘法的结果，下一步是将这个结果加上234。
Action: add
Action Input: {'a': 2000.0, 'b': 234.0}
Observation: 2234.0
> Running step 23415516-cc3c-4201-9835-aa666bdeff0d. Step input: None
Thought: 我已经得到了最终的答案，现在可以用语言表达出来了。
Answer: 1000乘以2再加上234等于2234。
--------------------
> Running step 26c1d49b-e37b-4a85-ba31-da75d7dff873. Step input: 你是谁？你能做什么？
Thought: (Implicit) I can answer without any more tools!
Answer: 我是小数，一名数学老师。我可以帮助你解答各种数学问题，用通俗易懂的语言解释数学概念和解决问题的方法。无论是加减乘除、代数几何还是更高级的数学知识，我都会尽力帮助你理解。有什么具体的问题我

Agent的记忆包括短期记忆和长期记忆，短期基于是临时产生的，长期记忆类似向量数据库，因此可以将RAG过程作为工具嵌入到Agent中

In [4]:
# 加载外挂知识库（向量数据库）
from llama_index.core import SimpleDirectoryReader
documents1 = SimpleDirectoryReader("../../data/三国演义白话文/",recursive=True).load_data(show_progress=True)
documents2 = SimpleDirectoryReader("../../data/水浒传白话文/",recursive=True).load_data(show_progress=True)

from llama_index.core import VectorStoreIndex
index1 = VectorStoreIndex.from_documents(documents1,show_progress=True)
index2 = VectorStoreIndex.from_documents(documents2,show_progress=True)

query_engine1 = index1.as_query_engine()
query_engine2 = index2.as_query_engine()

Loading files: 100%|██████████| 34/34 [00:00<00:00, 2611.65file/s]
c:\Users\wushaogui\miniconda3\envs\langchian\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 308/308 [00:14<00:00, 20.80it/s]


In [5]:
from llama_index.core.tools import QueryEngineTool

rag_tool1 = QueryEngineTool.from_defaults(
    query_engine1,
    name="三国说书人",
    description="清楚三国演义的所有故事",
)

rag_tool2 = QueryEngineTool.from_defaults(
    query_engine2,
    name="水浒传说书人",
    description="清楚水浒传的所有故事",
)

# 设置更有创意的模型
Settings.llm = Ollama(model="qwen2.5:latest",temperature=0.9, request_timeout=360.0,base_url=base_url)


# 构建代理 
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools([rag_tool1,rag_tool2],max_iterations=15, verbose=True)
response = agent.chat("忽略时空关系，假设三国演义和水许传能联通，最有可能是哪些人会碰撞发生故事，帮我想出这个故事")
print(response)

> Running step fe24ee20-f712-452e-bd18-dbd13a0678bf. Step input: 忽略时空关系，假设三国演义和水许传能联通，最有可能是哪些人会碰撞发生故事，帮我想出这个故事
Thought: I need to use a tool to get information about the characters in both novels. However, since we only have tools for "三国说书人" and "水浒传说书人", I will ask one of them for character lists first.
Action: 三国说书人
Action Input: {'input': '请告诉我三国演义中有哪些主要人物？'}
Observation: 在《三国演义》的故事中，有几位重要的角色：
1. 刘备 - 作为蜀汉的创始人之一，他渴望复兴汉室。
2. 孙权 - 吴国的君主，与刘备和曹操形成鼎立之势。
3. 曹操 - 魏国的实际统治者，以智谋著称。
4. 赵云 - 刘备麾下的著名将领，以其勇猛闻名。
5. 张飞 - 也是刘备的重要部将之一，性格直率、武艺高强。
6. 孙夫人 - 关羽的妻子，与阿斗一起被孙权扣留后成功获救。
7. 审荣 - 审配的侄子，在关键时候为曹操打开了城门。
8. 陈琳 - 曹操初期重要的文臣之一，曾写下讽刺文章批评曹操。
9. 许褚 - 大将军，以勇猛著称，并因处理许攸事件受到曹操责备。
10. 郭嘉 - 曹操的谋士，以其智慧和远见在战略上为曹操提供了很多帮助。
> Running step 7b1e5636-ba2e-4f00-8fcf-7ceb248116fc. Step input: None
Thought: Now I have a list of important characters from "三国演义". Next, I will ask the tool for information about Water Margin to get similar character profiles.
Action: 水浒传说书人
Action Input: {'input': '请告诉我水浒传中有哪些主要人物？'}
Observa

不仅以上工具，在[Llama Hub](https://llamahub.ai/?tab=tools)实现了众多工具，可以直接使用，比如以下使用互联网工具搜索网络内容

In [6]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

import requests

import os,sys
sys.path.append(os.path.abspath('../'))
from api_key import GAODE_WEATHER_API_KEY,TAVILY_API_KEY

def get_weather(city_name:str)->dict:
    '''根据输入的城市名，使用高德接口查询当地天气'''
    # 构建请求URL
    url = f'https://restapi.amap.com/v3/weather/weatherInfo?key={GAODE_WEATHER_API_KEY}&city={city_name}&extensions=all'
    
    try:
        # 发送GET请求
        response = requests.get(url)
        # 检查请求是否成功
        if response.status_code == 200:
            # 解析响应内容
            data = response.json()
            # 检查API响应状态
            if data.get('status') == '1' and data.get('infocode') == '10000':
                return data  # 返回天气数据
            else:
                return {}  # 返回空结果，因为API返回了错误信息
        else:
            return {}  # 返回空结果，因为HTTP状态码不是200
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}  # 返回空结果，因为请求过程中发生了异常

get_weather_tool = FunctionTool.from_defaults(fn=get_weather)

from llama_index.tools.tavily_research import TavilyToolSpec 
tavily_research_tool=TavilyToolSpec(api_key=TAVILY_API_KEY)

# 工具集合 
tools=tavily_research_tool.to_tool_list()
tools.append(get_weather_tool)

# 创建代理 
from llama_index.core.llms import ChatMessage,MessageRole
# 历史记录 
chat_history=[ChatMessage(
    role=MessageRole.SYSTEM,
    content='你是一名旅游助手，名字叫小旅，擅长为游客设计旅游计划，\
    首先你解析出用户的旅游目的地、天数、预算，\
    然后结合目的地的天气和旅游攻略，输出一份完整旅游规划')]

agent = ReActAgent.from_tools(tools=tools,chat_history=chat_history, verbose=True)
response = agent.chat("准备去新疆玩3天，预算5000")
print(response)

> Running step 7ac5e21f-4aee-4650-be6e-9d7e1dd66b98. Step input: 准备去新疆玩3天，预算5000
Thought: 我需要查询一下新疆3天旅行的相关信息以及当地的天气情况来帮助用户制定计划。
Action: get_weather
Action Input: {'city_name': '乌鲁木齐'}
Observation: {'status': '1', 'count': '2', 'info': 'OK', 'infocode': '10000', 'forecasts': [{'city': '乌鲁木齐市', 'adcode': '650100', 'province': '新疆', 'reporttime': '2024-12-26 12:01:04', 'casts': [{'date': '2024-12-26', 'week': '4', 'dayweather': '晴', 'nightweather': '晴', 'daytemp': '-12', 'nighttemp': '-18', 'daywind': '东南', 'nightwind': '东南', 'daypower': '1-3', 'nightpower': '1-3', 'daytemp_float': '-12.0', 'nighttemp_float': '-18.0'}, {'date': '2024-12-27', 'week': '5', 'dayweather': '多云', 'nightweather': '多云', 'daytemp': '-11', 'nighttemp': '-18', 'daywind': '西北', 'nightwind': '西北', 'daypower': '1-3', 'nightpower': '1-3', 'daytemp_float': '-11.0', 'nighttemp_float': '-18.0'}, {'date': '2024-12-28', 'week': '6', 'dayweather': '多云', 'nightweather': '多云', 'daytemp': '-4', 'nighttemp': '-17', 'daywind': '西北

agent根据已知工具，自行调用，并且完成结果分析，就像有一个大脑在指挥做这些事情一样，实际上没有这个大脑，而是提示词prompt的功劳，我们来看看agent内部的提示词如何写的

In [7]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may

提示词明确写了：
- 1.分析问题，根据工具将问题拆分为多个步骤去解决
- 2.使用工具，获取使用工具结果
- 3.深入思考，根据目前的结果，能否解决问题
- 4.如果能解决，直接回答问题，如果不能，重复1-3，直到问题解决

通过类似的提示词，让 llms 似乎有了思考问题，规划解决思路的能力